## Thesis Scripts:
This notebook is prepared by Aashish while working in his thesis project at Wageningen University and Research. His thesis topic is "Genetic variants in gene related to milk oligosaccharides content in a diverse pig population".

##### Supervisors: Henk Bovenhuis and Martijn Derks

##### Date: Feb'2023 - July' 2023

Data : "vcf" file of around more than 400 pigs (including non-sus species).
Tools mainly used: PLINK, bcftools, vcftools, Beagle, geneHapR. He constructed haplotype network using popart.

### Project methodology:
1) Identify candidate genes from other species via detailed literature review (GWAS, Transcriptomics) and come up with a list of genes that might be responsible for oligosaccharides synthesis in pig milk.

2) Do Principal component analysis to see population structure using vcf files.

3) Parse vcf information, subsetting vcf based on samples and genes.

4) Predict the consequence of variants (Missense, high impact variants and those variants that have been shown to effect phenotype in literature)

5) Calculate the allele frequency for each variants across population and within different breeds.

### Downloading tools and softwares

#### Tool used : Conda/Anaconda

Miniconda is a free minimal installer for conda. It is a small, bootstrap version of Anaconda that includes only conda, Python, the packages they depend on, and a small number of other useful packages, including pip, zlib and a few others. Use the conda install command to install 720+ additional conda packages from the Anaconda repository.

##### To download Miniconda3 into home directory.


In [ ]:
$ wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
$ ./Miniconda3-latest-Linux-x86_64.sh #or $ bash Miniconda3-latest-Linux-x86_64.sh

For more details on Conda, visit:
1) https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html

2) https://docs.conda.io/projects/conda/en/latest/user-guide/cheatsheet.html

### Principal Component Analysis
#### Tool used : Plink and R

All the codes that are given below are for the HPC annuna terminal. First, PLINK module should be loaded into the HPC.

In [ ]:
$ module load plink/1.9-180913

Once, plink is loaded into the server it can be used to run plink. The below code is for generating "eigenvec" and "eigenvalue" files. The input file is compressed "vcf file". 

In [ ]:
$ plink --vcf <sample.vcf.gz> --double-id –pca

The above code will generate .eigenvec and .eigenval files. Next step is to plot PC1 and PC2 using R. Below is the R script to make PCA plot out of .eigenvec and .eigenval files that was generated using PLINK.
#### Important:
1) Modify your "sample.eigenvec" file by adding factors (for eg: origin, breeds) on the last column(23rd) to make PCA clustering based on factor that you want to cluster. 
2) The input file is given as argument (below in code). That means we need to run additional code with below R script and "sample.eigenvec" file which is modified by including factors in 23rd column of eigenvec file.

In [ ]:
##This is a sample.R Rscript file

#!/usr/bin/env Rscript 
pdf("PCA_pruned.pdf") 
args = commandArgs(trailingOnly=TRUE)
pca <- read.table(args[1], row.names=1)
pca
col.rainbow <- rainbow(7)
palette(col.rainbow)
family <- as.factor(pca[,22])
plot(pca$V3, pca$V4,,col=family, pch=19, cex=0.4, xlab="", ylab="")
legend("bottomright", col=unique(family), legend=unique(family), pch=19)
title(main="PCA Cobb", xlab="PC1", ylab="PC2")
text(pca$V3, pca$V4, rownames(pca), cex=0.2)  ## to add sample names  in points of plot
dev.off()


Finally, run these codes below. 
Note: sample.R file is the file R script file with .R extension and modified.eigenvec is the eigenvec file with added column. 

In [ ]:
$ module load R

$ Rscript <sample.R> <modified.eigenvec>

Alternatively, I also have a code to plot PCA result using R but on CSV file. 

In [ ]:
library(tidyverse)
```


# read in result files
```{r}
eigenValues <- read.delim("plink.eigenval", delim = " ", col_names = F, show_col_types = FALSE)
view(eigenValues)
eigenValue <- read_csv("Without_NA.csv")  ### This file contains column with Animal ID, PC1, PC2, Breed and Origin
view(eigenValue)

```

# proportion of variance captured by each component
```{r}
pve <- round((eigenValues / (sum(eigenValues))*100), 2)  ### This provides the % of variation explained by each PCs

```

#PCA plot on above csv file
```{r}
eigenValue %>% 
  ggplot(aes(PC1, PC2, colour = Origin))+
  geom_point(size = 1, show.legend = TRUE) +
  geom_hline(yintercept = -0.0065, linetype="dotted") +
  geom_vline(xintercept = -0.00225, linetype="dotted") +
  labs(title = "PCA",
       x = paste0("PC1 (", round(pve[1,1],2)," %)"),
       y = paste0("PC2 (", round(pve[2,1],2)," %)"))
theme_minimal()
```

### Parsing VCF File
#### Tool used: bcftools , ensemblvep, PyVCF

In [ ]:
#!/bin/bash
#author: Aashish Gyawali
#purpose: Script pipeline for thesis analysis
#SBATCH --time=200
#SBATCH -n 1
#SBATCH -c 1
#SBATCH --error=error_pipeline.txt
#SBATCH --job-name=Thesis_Aashish
#SBATCH --mem=10000
#SBATCH --mail-user=aashish.gyawali@wur.nl

#exporting the plugins to update the vcf file for the allele frequency

export BCFTOOLS_PLUGINS=/home/WUR/gyawa001/miniconda3/libexec/bcftools/  # this is important to update AF in subset of VCF file that will be later created using "bcftools/1.17"

#to subset VCF file with whole sample into file containing only asian wild samples.
bcftools view -S asianwild_samples.txt Sscrofa11.1_Chr6.vcf.gz | bcftools +fill-tags | bgzip -c > asianwild_Chr6.vcf.gz

# to index the subsetted vcf file
bcftools index asianwild_Chr6.vcf.gz

# to get the vcf for certain region of genome. Here shows the example of FUT1
bcftools view -O v -e 'AC=0' -r 6:54034166-54081014 asianwild_Chr6.vcf.gz > asianwild_FUT1.vcf.gz

echo "Asian Wild done"

#same for rest of samples:
bcftools view -S asiandom_samples.txt Sscrofa11.1_Chr6.vcf.gz | bcftools +fill-tags | bgzip -c > asiandom_Chr6.vcf.gz
bcftools index asiandom_Chr6.vcf.gz
bcftools view -O v -e 'AC=0' -r 6:54076931-54081014 asiandom_Chr6.vcf.gz > asiandom_FUT1.vcf.gz

echo "Asian Domestic done"
# and so on

# to run vep first we need to activate conda environment
conda activate vep
module load vep

#running vep for each of the vcf file
#this will generate vep file and html summary file for each genes and population

vep -i asianwild_FUT1.vcf.gz --offline --dir /lustre/nobackup/WUR/ABGC/gyawa001/cache --force_overwrite --species sus_scrofa --pick --vcf --sift b -o asianwild_FUT1.vep.vcf
vep -i asiandom_FUT1.vcf.gz --offline --dir /lustre/nobackup/WUR/ABGC/gyawa001/cache --force_overwrite --species sus_scrofa --pick --vcf --sift b -o asiandom_FUT1.vep.vcf

echo "vep done"

echo "parsing information from vep"

module load python/3.9

# parse_genes_variants_WGS.py is a python code to parse information with Allele frequency per group and asianwild_FUT1.vep.vcf is a vcf file with vep done already from above code.

python parse_genes_variants_WGS.py asianwild_FUT1.vep.vcf > asianwild_FUT1.txt
python parse_genes_variants_WGS.py asiandom_FUT1.vep.vcf > asiandom_FUT1.txt

Script for parse_genes_variants_WGS.py

In [ ]:
## Script to parse variants in a VCF file 
## Install vcf,intermine, and subprocess package using: 
## 1. pip install PyVCF --user
## 2. pip install intermine --user
## 3. pip install subprocess --user

import sys
import vcf
from intermine.webservice import Service
from subprocess import Popen, PIPE
import subprocess
import pysam

def sample_dic():
    """ Read sample breed information """	
    sample_dic={}
    species_list = open("/lustre/nobackup/WUR/ABGC/gyawa001/final_species_list.tsv","r") #location and file name of sample list
    header = species_list.readline()
    for sample in species_list:
        sample = sample.strip().split("\t")
        samplename, origin, race_species, line_country, pigid = sample[0],sample[1],sample[2],sample[3],sample[4]
        sample_dic[samplename] = [origin.strip(), race_species.strip(), line_country.strip()]
    return sample_dic
    
def get_vcf_information(sample_dic):
    """ Read VCF file and parse output """
    vcf_reader = vcf.Reader(filename=sys.argv[1], strict_whitespace=True)
    print("Chr\tPos\tRef\tAlt\tRef_Homozygotes\tHeterozygotes\tAlt_Homozygotes\tMissing\tRef_Hom_samples\tHet_samples\tAlt_Hom_samples\tMissing_samples\tBreed\tCSQ\tIMPACT\tAF\tBreed_AF")
    gene_go_dic={}
    mgi_dic={}
    
    for record in vcf_reader:
        if int(record.INFO['AC'][0]) < 1:
            continue
        breed_dic={}
        freq_dic={}
        breed_count={}
        hom_ref_samples,het_samples,hom_samples,mis_samples=[],[],[],[]
        for sample in record.get_hom_refs():
            hom_ref_samples.append(sample.sample)
            breed = "-".join(sample_dic[sample.sample][1:])
            if breed in breed_count:
                breed_count[breed] +=2
            else:
                breed_count[breed] = 2
                
        for sample in record.get_hets():
            het_samples.append(sample.sample)
            breed = "-".join(sample_dic[sample.sample][1:]) ## Normal
            #breed = sample_dic[sample.sample][2] ## origin
            if breed in breed_dic:
                breed_dic[breed] +=1
                freq_dic[breed] +=1
            else:
                breed_dic[breed] = 1
                freq_dic[breed] =1
            if breed in breed_count:
                breed_count[breed] +=2
            else:
                breed_count[breed] = 2
                
        for sample in record.get_hom_alts():
            hom_samples.append(sample.sample)
            breed = "-".join(sample_dic[sample.sample][1:]) ## Normal
            #breed = sample_dic[sample.sample][2] ## origin
            if breed in breed_dic:
                breed_dic[breed] +=1
                freq_dic[breed] +=2
            else:
                breed_dic[breed] = 1
                freq_dic[breed] = 2
            if breed in breed_count:
                breed_count[breed] +=2
            else:
                breed_count[breed] = 2
                
        for sample in record.get_unknowns():
            mis_samples.append(sample.sample)
        if het_samples == []:
            het_samples = ["-"]
        if hom_samples == []:
            hom_samples = ["-"]
        if mis_samples == []:
            mis_samples = ["-"]
        csq=record.INFO['CSQ'][0].split("|")
        csq_impact=csq[2]
        sym,geneid=csq[3],csq[4]
        
        ## get GO annotation
        #go, gene_go_dic = gene_ontology(geneid,gene_go_dic)
        #go_output = geneid+":"+go
        
        ## Get phenotypes
        #phenotypes, mgi_dic = get_homologous_phenotypes(geneid, mgi_dic)
        #phenotypes=phenotypes.split("\t")
        #if phenotypes=="":
        #    phenotypes=["-","-","-"]	        
            
        ## get frequencies per breed
        freq_list=[]
        #pCADD_score = pCADD(record.CHROM,record.POS,record.ALT[0])	
        for breed in freq_dic:
            freq_list.append(breed+":"+str(round(float(freq_dic[breed])/breed_count[breed],2)))
        breed = ", ".join(["=".join([key, str(val)]) for key, val in breed_dic.items()])	
        outlist= [record.CHROM,record.POS,record.REF,record.ALT,record.num_hom_ref,record.num_het, record.num_hom_alt, record.num_unknown, ",".join(hom_ref_samples), ",".join(het_samples), ",".join(hom_samples), ",".join(mis_samples), breed, "".join(record.INFO['CSQ']), csq_impact, record.INFO['AF'][0], ",".join(freq_list)]
        print("\t".join(map(str,outlist)))
    
sample_dic=sample_dic()
get_vcf_information(sample_dic)

### Minor Allele Frequency and Density plot in R



In [ ]:
 vcftools --gzvcf allsamples_FUT1.vcf.gz --freq --out MAF_fut1

    # find minor allele frequency from all allele freq
R2<- read.csv("MAF_FUT2.csv")
library(ggplot2)
library(dplyr)
library(tidyverse)




#ploting maf
a <- ggplot(R2, aes(maf)) + geom_density(fill = "dodgerblue1", colour = "red", alpha = 0.5, adjust = 80)
a + theme_light()



### Haplotype Analysis

#### Tool used : Beagle/5.2 , geneHapR Package (R), PopArt

The first step in haplotype analysis is phasing of genotypes.  The VCF file has "/" symbol in GT format which means the genotype is Unphased.
What does Phasing means?:
So, we need to phase the genotype first and make it in "|" symbol.

The current version I am using for the phasing of genotypes is Beagle/5.2. However, newer versions have been already available. Beagle  has error when there are missing genotypes in the samples which is stated as "/.:" and it cannot work with missing genotypes. So I needed to convert all the missing genotypes into 0/0 using sed command as below:

In [7]:
 cat allsamples_Chr6.vcf | sed -i 's/\./0\/0/g' > allsamples_Chr6.replaced.vcf.gz

cat: allsamples_Chr6.vcf: No such file or directory
sed: no input files


This will change all . into 0/0 including . of ID even. haha. Next we will run a code to phase the genotypes of our 6th chromosome.

In [ ]:
java -jar /lustre/nobackup/WUR/ABGC/gyawa001/Beagle/beagle.28Jun21.220.jar
 gt=Sscrofa11.1_Chr6.replaced.vcf out=output

#### Counting number of haplotypes and constructing haplotype network for FUT genes
Package "geneHapR" was used to make this analysis with some manual work in Excel.

In [ ]:
#“Introduction of ‘geneHapR’
# Haplotype analysis

library(adegenet)
library(pegas)
library(vcfR)

#to read a vcf file
vcf <- read.vcfR("allsamples_FUT1_phased_edited.vcf")

#to extract haplotypes
haplotypes <- extract.haps(vcf)

#to write haplotypes into csv file and save it in local computer
write.csv(haplotypes, "allsamples_FUTs.csv", row.names = F)

#the above steps are needed to do some editing in excel.
#after that geneHapR can be used.


#install package named "geneHapR"

library(geneHapR)

#import vcf file


vcf <- import_vcf("allsamples_FUT1_phased_edited.vcf")
View(vcf)
AccINFO <- import_AccINFO("AccINFO.txt")
View(AccINFO)
#haplotype calculation

hapResult <- vcf2hap(vcf, hapPrefix = "H" , hetero_remove = T, na_drop = T)
hapResult
View(hapResult)
# Chech number of sites conclude in hapResult
sites(hapResult)

#Summary hapResult
hapSummary <- hap_summary(hapResult)
hapSummary
View(hapSummary)
write.csv(hapSummary, "HapSummary_FUT1.csv", row.names = T)

hapSummary_removing_singletons <- hapSummary[1:44, ]
#Visualize haplotye as table
plotHapTable(hapSummary_removing_singletons)

#hapnet calculation and Visualization
hapNet <- get_hapNet(hapSummary_removing_singletons,
                     AccINFO = AccINFO,
                     groupName = "Origin")

# plot haploNet

plotHapNet(hapNet,
           size = "freq",                   # circle size
           scale = "log2",                 # scale circle with 'log10(size + 1)'
           cex = 0.2,                       # size of hap symbol
           col.link = 2,                    # link colors
           link.width = 2,                  # link widths
           show.mutation = 2,               # mutation types one of c(0,1,2,3)
           legend = c(14.5, 6.5))        # legend position



## Linkage Disequilibrium calculation

To calculated LD between two markers was done. The first marker was the the significant marker from LIFT eQTL analysis of 4 tissues from 100 pigs samples. The signal was very high for lung tissue for FUT2 gene. To see if there is any relation between the significant SNP and the haplotype first linkage disequilibrium is carried out. We expect, the haplotype of FUT2 might have a some relation with expression signal from eQTL. There were 12 significant SNPs with same p,q values. We selected *6:53685920* as the SNP for eQTL and the LD between this SNP and the SNP marker in FUT2 haplotype was calculated.

The important things to be consider are:
1) First the VCF file must be changed into format recognised by PLINK. The generated file will be with .bim and .fam extension,

2) Than, since our vcf file didn't have unique IDs, the IDs need to be given for each variants so later LD between SNPs based on variant ID can be calculated.

3) LD calculation between variants.


In [ ]:
plink --vcf your_file.vcf --make-bed --out output_file

There is no variant ID is the VCF file I am using. Thus I need to set the variant IDs for all the variants that are available in the output_file. The command below will allow to set the variants ID based on their location. This makes easier to calculate the LD between two SNPs.

In [ ]:
module load plink/2
plink2 --bfile output_file --set-var-ids @:# --make-bed --out updated_file

The last step of calculating the LD between two SNPs.

In [ ]:
plink --bfile allsamples_Chr6_updated --ld 6:53685920 6:54034275 --r2 

The LD results consists of R-sq, D' and In phase alleles as shown below

LD result:
   R-sq = 0.635214       D' = 0.936853
